In [ ]:
# Get the data from Cadena et al. 2019
# NOTE: they applied to their images, shown to the monkeys, a circular aperture mask with cosine fade-out
import pickle
import numpy as np
import matplotlib.pyplot as plt

with open('../../data/cadena_plosCB19/cadena_ploscb_data.pkl', 'rb') as g:
    loaded_data = pickle.load(g)

# Cadena found that the transfer‐learning approach (using VGG features) reached near-optimal performance with as little as 20% of the available data
SAMPLE_SIZE = 1400
rng = np.random.RandomState(42)
SAMPLE_INDICES = np.random.choice(loaded_data["images"].shape[0], SAMPLE_SIZE, replace=False)

# Check the available keys and shapes
for key, value in loaded_data.items():
    print(f"{key}: {value.shape if isinstance(value, np.ndarray) else type(value)}")

In [ ]:
# Preprocess neural data

responses = loaded_data["responses"]
print("Responses shape:", responses.shape) # (4, 7250, 166) -> 4 repetitions, 7250 images, 166 neurons

# Check how many images have no valid responses across all neurons and repetitions
fully_missing_images = np.isnan(loaded_data["responses"]).all(axis=(0, 2)) # Check over repetitions & neurons
num_fully_missing = fully_missing_images.sum()
print(f"Fully missing images: {num_fully_missing}/{loaded_data['responses'].shape[1]}")

# Load neural responses and average over the 4 repetitions
averaged_responses = np.nanmean(responses, axis=0) # Shape: (7250, 166)
print("Averaged Responses shape:", averaged_responses.shape)

averaged_responses_cleaned = np.nan_to_num(averaged_responses) # Apply Cadena's preprocessing: Replace NaNs with 0
print('Responses sample:', averaged_responses_cleaned[:20])

# Select only the first 3625 samples, because we will only use half the images
processed_neural_responses = averaged_responses_cleaned[SAMPLE_INDICES]
print('Final responses shape:', processed_neural_responses.shape)

In [ ]:
# # Prepare images for VGG-19

# import torchvision.transforms as transforms
# from PIL import Image
# import numpy as np

# # Define VGG-compatible transformation
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),  
#     transforms.Grayscale(num_output_channels=3), # Convert grayscale to 3 channels
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# # Apply transformation to all images (using half for now due to memory constraints)
# sample = loaded_data["images"][SAMPLE_INDICES]
# images = np.stack([
#     transform(Image.fromarray(img.astype(np.uint8))).numpy() for img in sample
# ])

# print("Transformed Images Shape:", images.shape)  # (7250, 3, 224, 224)

In [ ]:
### Cadena's preprocessing

import numpy as np
import cv2
from PIL import Image
import torchvision.transforms as transforms

# Assume you have a function that implements Cadena's preprocessing:
def preprocess_image(img, global_avg_std):
    # --- Contrast matching ---
    h, w = img.shape  # raw image should be 140x140
    patch_size = 70
    start_row_patch = (h - patch_size) // 2
    start_col_patch = (w - patch_size) // 2
    central_patch = img[start_row_patch:start_row_patch+patch_size, start_col_patch:start_col_patch+patch_size]
    current_mean = np.mean(central_patch)
    current_std = np.std(central_patch)
    if current_std == 0:
        current_std = 1.0
    matched_img = (img - current_mean) * (global_avg_std / current_std) + 128
    matched_img = np.clip(matched_img, 0, 255).astype(np.uint8)
    
    # --- Crop the central 80x80 region ---
    crop_size = 80
    start_row_crop = (h - crop_size) // 2
    start_col_crop = (w - crop_size) // 2
    cropped_img = matched_img[start_row_crop:start_row_crop+crop_size, start_col_crop:start_col_crop+crop_size]
    
    # --- Downsample the crop to 40x40 using bicubic interpolation ---
    resized_img = cv2.resize(cropped_img, (40, 40), interpolation=cv2.INTER_CUBIC)
    
    # --- Z-score the image ---
    mean_val = np.mean(resized_img)
    std_val = np.std(resized_img)
    if std_val == 0:
        std_val = 1.0
    zscored_img = (resized_img - mean_val) / std_val
    return zscored_img

def compute_global_avg_std(images):
    patch_size = 70
    stds = []
    h, w = images[0].shape
    start_row = (h - patch_size) // 2
    start_col = (w - patch_size) // 2
    for img in images:
        central_patch = img[start_row:start_row+patch_size, start_col:start_col+patch_size]
        stds.append(np.std(central_patch))
    return np.mean(stds)

# Suppose raw_images is your loaded array of shape (n_images, 140, 140)
# Cadena found that the transfer‐learning approach (using VGG features) reached near-optimal performance with as little as 20% of the available data
SAMPLE_SIZE = 1400
rng = np.random.RandomState(42)
SAMPLE_INDICES = np.random.choice(loaded_data["images"].shape[0], SAMPLE_SIZE, replace=False)
raw_images = loaded_data["images"][SAMPLE_INDICES]
global_avg_std = compute_global_avg_std(raw_images)

# Apply the preprocessing to a subset (or all) of your images:
preprocessed_images = np.array([preprocess_image(img, global_avg_std) for img in raw_images])
print("Preprocessed images shape:", preprocessed_images.shape)  # Expect (n, 40, 40)

# Now, if your model expects 224x224 images, you can upsample:
upsampled_images = np.array([cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC) for img in preprocessed_images])
print("Upsampled images shape:", upsampled_images.shape)  # (n, 224, 224)

# Then convert to PIL images and apply the VGG-compatible transform:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # if needed, though upsampled_images are already grayscale
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transformed_images = np.stack([transform(Image.fromarray(img.astype(np.uint8))).numpy() for img in upsampled_images])
print("Transformed Images Shape:", transformed_images.shape)  # Should be (n, 3, 224, 224)

In [ ]:
import torch
import torchvision.models as models
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import os

PATH_TO_FEATURES = "../../data/cadena_plosCB19/vgg_features_preprocessed.npy"

if os.path.exists(PATH_TO_FEATURES):
    print(f"Features file '{PATH_TO_FEATURES}' exists. Loading features...")
else:
    print(f"Features file does not exist. Extracting features...")
    
    # Define feature extractor (VGG-19 up to conv3_1)
    class VGGFeatureExtractor(torch.nn.Module):
        def __init__(self):
            super(VGGFeatureExtractor, self).__init__()
            # Extract feature maps (256, 56, 56) from conv3_1 layer (BEFORE max pooling)
            self.features = torch.nn.Sequential(*list(models.vgg19(pretrained=True).features.children())[:14])

        def forward(self, x):
            return self.features(x)

    # Device configuration (Use GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    feature_extractor = VGGFeatureExtractor().to(device).eval()

    # Convert images to Torch tensor dataset
    images_tensor = torch.tensor(transformed_images, dtype=torch.float32).to(device)  # Shape: (3625, 3, 224, 224)

    # DataLoader for batch processing
    batch_size = 8
    dataset = TensorDataset(images_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Store all batches in memory
    all_batches = []

    # Extract features batch-wise and store in RAM
    with torch.no_grad():  # Disable autograd for efficiency
        for batch_idx, (batch,) in enumerate(dataloader):
            batch = batch.to(device)  # Move batch to GPU (if available)
            features = feature_extractor(batch).cpu().numpy()  # Move to CPU

            all_batches.append(features)  # Store batch in memory

            # Free memory
            del batch, features
            torch.cuda.empty_cache()

            print(f"Processed batch {batch_idx + 1}/{len(dataloader)}")

    # Concatenate all batches into a single NumPy array
    vgg_features = np.concatenate(all_batches, axis=0)
    print("Final Features Shape:", vgg_features.shape) # Expected: (3625, 256, 56, 56)
    np.save(PATH_TO_FEATURES, vgg_features)

In [ ]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# Load features from file (assumed shape: (1400, 256, 56, 56))
PATH_TO_FEATURES = "../../data/cadena_plosCB19/vgg_features_preprocessed.npy"
vgg_features = np.load(PATH_TO_FEATURES, mmap_mode="r")
# Flatten features: (1400, feature_dim)
X = vgg_features.reshape(vgg_features.shape[0], -1)

# processed_neural_responses: averaged responses for selected neurons (shape: (1400, 166))
y = processed_neural_responses

print("Final Shapes -> X:", X.shape, "y:", y.shape)

# --- Create train/validation/test splits while keeping track of indices ---
n_total = y.shape[0]  # total number of images used (should be 1400)
indices = np.arange(n_total)

# First, split into train+val (80%) and test (20%)
train_val_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42, shuffle=True)
X_train_val = X[train_val_idx]
X_test = X[test_idx]
Y_train_val = y[train_val_idx]
Y_test = y[test_idx]

# Then, split train+val into training (80% of 80% = 64% overall) and validation (16% overall)
train_idx, val_idx = train_test_split(train_val_idx, test_size=0.2, random_state=42, shuffle=True)
X_train = X[train_idx]
X_val = X[val_idx]
Y_train = y[train_idx]
Y_val = y[val_idx]

print("Split Shapes:")
print("  X_train:", X_train.shape, "Y_train:", Y_train.shape)
print("  X_val  :", X_val.shape,   "Y_val  :", Y_val.shape)
print("  X_test :", X_test.shape,  "Y_test :", Y_test.shape)

# --- Fit Ridge Regression ---
alpha = 100000
ridge_reg = Ridge(alpha=alpha)
ridge_reg.fit(X_train, Y_train)
y_pred = ridge_reg.predict(X_test)
r2 = r2_score(Y_test, y_pred)
print(f"Test R² Score: {r2:.3f}")

# --- Compute FEV using trial-level responses ---
# For FEV, we need the original (trial-level) responses corresponding to the images used.
# Assume loaded_data["responses"] has shape (n_reps, total_images, total_neurons)
# and that processed_neural_responses was computed using the first 1400 images.
all_responses = loaded_data["responses"][:, :n_total, :]  # Restrict to the 1400 images used.
# Also restrict to the top neurons (if not already done); assume processed_neural_responses has the correct neurons.
all_responses_test = all_responses[:, test_idx, :]  # Now shape: (n_reps, n_test, 166)

def compute_fev(y_true, y_pred, all_responses, epsilon=1e-8):
    """
    y_true: Averaged neural responses, shape (n_samples, n_neurons)
    y_pred: Model predictions, shape (n_samples, n_neurons)
    all_responses: Trial-level responses for these images, shape (n_reps, n_samples, n_neurons)
    """
    # Total variance over test images for each neuron (from the averaged responses)
    total_variance = np.var(y_true, axis=0)
    # Variance explained by the predictions (per neuron)
    explained_variance = np.var(y_pred, axis=0)
    # Noise variance: compute variance over repetitions for each image and neuron, then average over images.
    noise_variance = np.nanmean(np.var(all_responses, axis=0), axis=0)
    
    fev_per_neuron = 1 - (total_variance - explained_variance) / (total_variance - noise_variance + epsilon)
    return np.mean(fev_per_neuron)

fev_score = compute_fev(Y_test, y_pred, all_responses_test)
print(f"Fraction of Explainable Variance (FEV): {fev_score:.3f}")

# --- Optionally, plot the regression performance ---
pcs_values = [0, 10, 100, 1000]
train_r2_scores = []
val_r2_scores = []
test_r2_scores = []
fev_scores = []

# For demonstration, loop over different PCA dimensionalities (if desired)
from sklearn.decomposition import PCA
for num_pcs in pcs_values:
    if num_pcs > 0:
        pca = PCA(n_components=num_pcs, random_state=42)
        X_train_pca = pca.fit_transform(X_train)
        X_val_pca   = pca.transform(X_val)
        X_test_pca  = pca.transform(X_test)
    else:
        X_train_pca = X_train
        X_val_pca   = X_val
        X_test_pca  = X_test
    
    reg = Ridge(alpha=alpha)
    reg.fit(X_train_pca, Y_train)
    
    train_pred = reg.predict(X_train_pca)
    val_pred   = reg.predict(X_val_pca)
    test_pred  = reg.predict(X_test_pca)
    
    train_r2 = r2_score(Y_train, train_pred)
    val_r2   = r2_score(Y_val, val_pred)
    test_r2  = r2_score(Y_test, test_pred)
    
    train_r2_scores.append(train_r2)
    val_r2_scores.append(val_r2)
    test_r2_scores.append(test_r2)
    
    # Compute FEV on test set
    # For this, we assume that the same test indices (test_idx) apply to the original responses.
    fev_val = compute_fev(Y_test, test_pred, all_responses_test)
    fev_scores.append(fev_val)
    
    print(f"PCs: {num_pcs}, Train R²: {train_r2:.4f}, Val R²: {val_r2:.4f}, Test R²: {test_r2:.4f}, Test FEV: {fev_val:.4f}")

plt.figure(figsize=(6, 4))
plt.plot(pcs_values, train_r2_scores, marker='o', linestyle='--', label='Train R²')
plt.plot(pcs_values, val_r2_scores, marker='s', linestyle='-', label='Validation R²')
plt.plot(pcs_values, test_r2_scores, marker='^', linestyle='-', label='Test R²')
plt.plot(pcs_values, fev_scores, marker='d', linestyle='-', label='Test FEV')
plt.xlabel("Number of Principal Components")
plt.ylabel("Score")
plt.title(f"Ridge Regression from {image_representation_name} to Neural Responses")
plt.legend()
plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
plt.show()

In [ ]:
# Fit a Poisson regression model

import numpy as np
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

vgg_features = np.load(PATH_TO_FEATURES, mmap_mode="r") # Use mmap_mode for large files

# X: (3625, num_features) and y: (3625, 166)
X = vgg_features.reshape(vgg_features.shape[0], -1) # Flatten from (3625, 256, 56, 56) (3625, feature_dimensions)
y = processed_neural_responses
print("Final Shapes -> X:", X.shape, "y:", processed_neural_responses.shape)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

n_neurons = y_train.shape[1]
models = []
y_pred_test_all = np.zeros_like(y_test)
r2_list = []

# Loop over each neuron (output dimension)
for i in range(1):
    # Fit a separate PoissonRegressor for neuron i
    model = PoissonRegressor(alpha=0.01, max_iter=1000)
    model.fit(X_train, y_train[:, i])
    models.append(model)
    
    # Predict on the test set for this neuron
    y_pred = model.predict(X_test)
    y_pred_test_all[:, i] = y_pred
    
    # Compute R² for this neuron and store it
    r2 = r2_score(y_test[:, i], y_pred)
    r2_list.append(r2)

# Compute mean R² across neurons
mean_r2 = np.mean(r2_list)
print(f"Mean Test R² Score: {mean_r2:.3f}")

# Optionally, define the FEV function and compute average FEV
def compute_fev(y_true, y_pred):
    # Compute noise variance, total variance, and explained variance for each neuron
    noise_variance = np.var(y_true - np.mean(y_true, axis=0), axis=0)
    total_variance = np.var(y_true, axis=0)
    explained_variance = np.var(y_pred, axis=0)
    fev = 1 - (total_variance - explained_variance) / (total_variance - noise_variance)
    return np.mean(fev)

fev_score = compute_fev(y_test, y_pred_test_all)
print(f"Mean Fraction of Explainable Variance (FEV): {fev_score:.3f}")